You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: Unveiling the Latest Trends and Key Players in Artificial Intelligence

Introduction:
- Brief overview of Artificial Intelligence (AI) and its impact on various industries
- Mention of the rapid advancements and innovations in the field of AI
- Introduction to the key players and companies leading the AI revolution

Key Points:
1. Latest Trends in AI:
- Natural Language Processing (NLP) and Conversational AI
- Computer Vision and Image Recognition
- AI in Healthcare and Medical Diagnosis
- Autonomous 

I now can give a great answer

Final Answer:
# Unveiling the Latest Trends and Key Players in Artificial Intelligence

## Introduction:
Artificial Intelligence (AI) has revolutionized various industries with its cutting-edge technology and innovative applications. The rapid advancements in AI have paved the way for transformative solutions in fields such as healthcare, finance, and transportation. Key players and companies are at the forefront of this AI revolution, driving progress and shaping the future of technology.

## Latest Trends in AI:
The latest trends in AI encompass a wide range of applications that are reshaping industries and society as a whole. Natural Language Processing (NLP) and Conversational AI are enabling more intuitive interactions between humans and machines. Computer Vision and Image Recognition are enhancing visual perception and analysis, leading to advancements in fields like autonomous vehicles and facial recognition technology. AI in Healthcare and Medical

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# Unveiling the Latest Trends and Key Players in Artificial Intelligence

## Introduction:
Artificial Intelligence (AI) has revolutionized various industries with its cutting-edge technology and innovative applications. The rapid advancements in AI have paved the way for transformative solutions in fields such as healthcare, finance, and transportation. Key players and companies are at the forefront of this AI revolution, driving progress and shaping the future of technology.

## Latest Trends in AI:
The latest trends in AI encompass a wide range of applications that are reshaping industries and society as a whole. Natural Language Processing (NLP) and Conversational AI are enabling more intuitive interactions between humans and machines. Computer Vision and Image Recognition are enhancing visual perception and analysis, leading to advancements in fields like autonomous vehicles and facial recognition technology. AI in Healthcare and Medical Diagnosis is revolutionizing patient care and diagnosis processes, improving outcomes and efficiency. Autonomous Vehicles and Robotics are leveraging AI technologies to enable self-driving cars and automated manufacturing processes. Additionally, AI Ethics and Bias Mitigation are crucial considerations to ensure fair and responsible AI implementations.

## Key Players in AI:
Leading the AI revolution are key players and companies that are driving innovation and shaping the future of technology. Google AI, with its deep learning algorithms and AI research initiatives, is pushing the boundaries of machine learning and AI applications. IBM Watson, known for its cognitive computing capabilities, is transforming industries with its AI-powered solutions. Amazon Web Services (AWS) is providing cloud-based AI services that enable businesses to leverage AI technologies at scale. Microsoft AI is pioneering advancements in AI research and development, with a focus on democratizing AI for all. OpenAI, a non-profit organization dedicated to AI safety and research, is pushing the boundaries of AI capabilities while prioritizing ethical considerations.

## Noteworthy News in AI:
Recent breakthroughs in AI research have led to exciting developments in AI applications across various industries. From healthcare to finance, AI is revolutionizing processes and driving innovation. The impact of AI on society and the workforce is profound, with implications for jobs, privacy, and ethical considerations. Staying updated on the latest AI news and advancements is crucial for tech enthusiasts, business professionals, students, and researchers to understand the evolving landscape of AI technology.

## Conclusion:
As AI continues to transform industries and society, it is essential to stay informed about the latest trends and key players shaping the future of technology. By exploring the latest advancements in AI and understanding the key players driving innovation, readers can gain valuable insights into the impact of AI on industries and society. Embracing AI technologies responsibly and ethically is key to unlocking the full potential of AI and ensuring a bright future for technology.

In conclusion, the blog post provides a comprehensive overview of the latest trends and key players in Artificial Intelligence. By incorporating SEO keywords naturally, structuring the post with engaging sections, and proofreading for grammatical errors, the content is tailored to appeal to tech enthusiasts, business professionals, students, and researchers. The call to action encourages readers to explore more about AI trends and key players, stay updated on AI news and advancements, and access relevant resources for further reading. Ultimately, the blog post aims to engage and empower readers to make informed decisions about AI technologies and their impact on industries and society.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [30]:
#topic = "YOUR TOPIC HERE"
topic = "Cicadas Arise and Sabotage the Village"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Cicadas Arise and Sabotage the Village.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer.

Final Answer: 

Content Plan: 
Title: The Invasion of Cicadas: How They Arise and Sabotage the Village

Outline:
I. Introduction
- Brief explanation of the phenomenon of cicadas emerging after years underground
- Mention of the impact they can have on vegetation and agriculture

II. Latest Trends and Key Players
- Discuss the recent surge in cicada populations and the potential threats they pose to villages
- Highlight the key players in the village affected by the cicada invasion, such as farmers and local 

I now can give a great answer

Final Answer:
# The Invasion of Cicadas: How They Arise and Sabotage the Village

Cicadas, those buzzing insects that emerge after years underground, are causing quite a stir in villages across the country. With their sheer numbers and voracious appetites, these insects can wreak havoc on vegetation and agriculture, posing a significant threat to farmers and local businesses. The phenomenon of cicadas rising from the ground is not new, but the recent surge in populations has brought renewed attention to the potential damage they can inflict.

In recent years, there has been a noticeable increase in cicada populations, leading to concerns among villagers about the impact they may have on their crops and livelihoods. Farmers and agricultural workers are at the forefront of this issue, as they are directly affected by the damage caused by cicadas. Local businesses that rely on agricultural products are also feeling the effects of these buzzing bugs, with pot

In [31]:
Markdown(result)

# The Invasion of Cicadas: How They Arise and Sabotage the Village

Cicadas, those buzzing insects that emerge after years underground, are causing quite a stir in villages across the country. With their sheer numbers and voracious appetites, these insects can wreak havoc on vegetation and agriculture, posing a significant threat to farmers and local businesses. The phenomenon of cicadas rising from the ground is not new, but the recent surge in populations has brought renewed attention to the potential damage they can inflict.

In recent years, there has been a noticeable increase in cicada populations, leading to concerns among villagers about the impact they may have on their crops and livelihoods. Farmers and agricultural workers are at the forefront of this issue, as they are directly affected by the damage caused by cicadas. Local businesses that rely on agricultural products are also feeling the effects of these buzzing bugs, with potential disruptions to their supply chains and revenues.

Reports of cicada sightings and infestations in nearby areas have been on the rise, prompting villages to take action to protect their crops and properties. While the thought of a cicada invasion may be daunting, there are success stories of villages that have effectively managed these infestations in the past. By implementing proactive measures and utilizing effective control methods, these villages have been able to minimize the damage caused by cicadas and safeguard their communities.

For farmers and agricultural workers who are concerned about the impact of cicadas on their crops, it is essential to stay informed about prevention and mitigation strategies. Residents of villages at risk of cicada infestations can benefit from learning how to identify and manage these insects to protect their properties and livelihoods. By understanding the lifecycle of cicadas and implementing control methods, villagers can effectively combat these buzzing bugs and minimize the potential damage they may cause.

As villages continue to face the threat of cicada invasions, it is crucial for residents to take proactive measures to protect their crops and properties. By staying informed about cicada identification and management, villagers can effectively combat these insects and safeguard their communities. For further information and resources on cicada prevention, readers are encouraged to explore the links provided in this article and take action to mitigate the impact of these buzzing bugs.

In conclusion, the invasion of cicadas poses a significant challenge to villages across the country, but with proactive measures and effective control methods, residents can successfully manage these infestations and protect their communities from the potential damage. By staying informed and taking action, villages can combat the threat of cicadas and ensure the sustainability of their crops and livelihoods.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).